# Anomaly Detection using a Variational Auto-Encoder
In this project, we are using a Variational Auto-Encoder (VAE) to detect anomalies in a timeseries.

## 1. Introduction to VAEs in Anomaly Detection
### 1.1 What is a VAE?
VAEs are a class of generative models that learn a compressed latent representation of the data. Intuitively, they learn the inherent structure of the data, so that it can be reconstructed just by providing a few values. As an analogy, imagine that you were asked to describe any object to someone just from basic shapes, so that they can draw it. It would take ages to explain everything in enough detail. Now, imaging you can tell them the object is in fact a dog. Immediately, it becomes much easier to explain, and within a few minutes, you should be able to give a pretty accurate description of the dog. This is because you and your friend share a common knowledge of what constitutes a dog and what distinguishes one dog from another. With this pre-learned knowledge, the description of the object becomes much more compact. 

To learn this structure, the VAE has two parts: An encoder and a decoder. The compact representation (the "latent vector") is sandwiched between these two parts. During training, we present the encoder with data samples that it converts into latent vectors. Then the decoder takes the latent vectors and tries to reconstruct the data samples. By minimising the recontruction error (plus a regularisation loss, more on that later), the encoder gets better at compressing the data into a meaningful latent vector, and the decoder becomes better at converting this latent vector into plausible reconstructed samples.

However, we also need to make sure that the VAE is able to reconstruct samples outside of the training set, i.e. to generalise. This is achieved by penalising ("regularising") the reconstruction loss with an additional term, the Kullback-Leibler Divergence. In broad terms, this ensures that the VAE learns the distribution of the data, rather than the datapoints themselves. There is much more to the theory of VAEs (e.g. the "reparameterisation trick") but this is outside the scope of this notebook. You can learn a lot more about VAEs [here](https://arxiv.org/abs/1606.05908) \[1\].

### 1.2 Why use a VAE in Anomaly Detection?

The idea behind Anomaly Detection is to detect samples that are far from what is usually seen, in some sense or other. The definition of "far" is the difficult bit. In simple, one-dimensional cases, we could just look at the value tracked over time and decide that extreme values are anomalies. Lots of methods use this idea in more or less sophisticated ways.

However, things become trickier in higher-dimensional spaces where variables interact with each other or are correlated in some non-obvious ways. Sure, a sheep is "far" from a dog, but how do you formalise this distance? Its size is clearly not enough: some dogs are as big or larger than sheeps. Colour doesn't work either -- there are black sheeps and white dogs. Ear shape -- let's not even go there. The decision will clearly require a combination of many variables.

This is where VAEs come in useful. As we saw, a VAE trained on dogs will have learned a latent, non-explicit representation of the structural features of a dog. What will happen if we pass it an instance of a sheep? It will try to reconstruct the sheep, but since the structure of a sheep is different to that of a dog, most likely the reconstruction will not be very good. As a result, the reconstruction loss will probably be quite poor on this particular sample, much worse than on dog instances -- at least if our VAE has benn properly trained. By detecting data samples that cause a large reconstruction loss, we can therefore hope to identify anomalies.

The process is as follows:  

- Gather and preprocess data, including train/test split,
- Build a VAE and train it on the training set,
- Pass test samples to the VAE and record the reconstruction loss for each,
- Identify test samples with a reconstruction loss higher than some criterion and flag them as anomalies.

### 1.3 Scope of this project
In this notebook, we will use one of the public datasets available on Kaggle: https://www.kaggle.com/boltzmannbrain/nab
It contains many one-dimensional timeseries, but to make things a bit more interesting, we will do some feature engineering and make the data multi-dimensional. This is also often useful in improving the accuracy of any model.

## 2. Data preprocessing

Much of the pre-processing in this section has been inspired by this great [notebook](https://www.kaggle.com/victorambonati/unsupervised-anomaly-detection) by Victor Ambonati.

Before we start, we need to install and import some useful libraries. For this project, I am using [Pytorch-Lightning](https://pytorch-lightning.readthedocs.io/en/latest/), a thin layer on top of Pytorch that handles a lot of the tedious tasks such as writing a training loop. The experiments are recorded using [Weights and Biases](https://app.wandb.ai/), but you can use whatever method you want, or none.

In [ ]:
! pip install pytorch-lightning

In [ ]:
! pip install matplotlib==3.1.3  # had issues with package incompatibilies

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import sklearn
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb

In [ ]:
print('Package versions:')
print('Pytorch:', torch.__version__)
print('Pytorch-Lightning:', pl.__version__)
print('Matplotlib:', matplotlib.__version__)
print('scikit-learn:', sklearn.__version__)
print('Weights&Biases:', wandb.__version__)

These are the available datasets:

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

We pick `machine_temperature_system_failure.csv`, but the pipeline below can apply to others too, with some minor adaptation. This data comes from a temperature sensor from an internal component of a large industrial machine.

In [ ]:
datafile_path = Path('/kaggle/input/nab/realKnownCause/realKnownCause/machine_temperature_system_failure.csv')
datasets_root = Path('/kaggle/working')

The data is monodimensional: It only has one "predictor" (`timestamp`) and the outcome (`value`):

In [ ]:
raw_dt = pd.read_csv(datafile_path)
raw_dt.head()

In [ ]:
raw_dt.describe()

In [ ]:
data = raw_dt.copy()
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.plot(x='timestamp', y='value', figsize=(12, 10));

We can already intuitively see a few of unusual values. We will see if our model will be able to pick them up... For reference, the paper that introduced this dataset can be found [here](https://reader.elsevier.com/reader/sd/pii/S0925231217309864?token=C53EC725CFFDDB81C89A60CA5CA9B4DDD0E1DBFC0F8975119E6CB3555C8B2D6AC0418BF5AE7AE721DA3BB77DDD638190) \[1\] and its figure 1 shows this timeseries with hand-labeled anomalies. The first anomaly, around Dec 16th, is a planned shutdown. The third anomaly, around Feb 8th, is a catastrophic failure. Around Jan 30th, a harder-to-detect anomaly indicated the onset of the problem that led to anomaly 3.

In [ ]:
data['timestamp'].min(), data['timestamp'].max()

We do not have much information about what kind of machine or industry we are dealing with, which is a bit of an issue when trying to use this data, especially when it comes to feature engineering. We will therefore have to make assumptions. 

The timestamps cover the Christmas and New Year holidays. Since we are dealing with an industrial machine, it stands to reason that its workload might be affected by holidays, and maybe even by the proximity (in time) of a holiday. In the absence of additional information, we are going to assume that the applicable holidays are those typical in Europe and the Americas, i.e. Christmas and New Year's Day. By the same reasoning, we might need to know the day of the week (possibly lower workload on weekends?) or the hour of the day. Again, we will assume that weekends are Satuday and Sunday. 

We can easily extract all this information from the timestamp.

In [ ]:
data['day'] = data['timestamp'].dt.day
data['month'] = data['timestamp'].dt.month
data['hour_min'] = data['timestamp'].dt.hour + data['timestamp'].dt.minute / 60

data['day_of_week'] = data['timestamp'].dt.dayofweek
data['holiday'] = 0
data.loc[(data['day'] == 25) & (data['month'] == 12),'holiday'] = 1  # Christmas
data.loc[(data['day'] == 1) & (data['month'] == 1),'holiday'] = 1  # New Year's Day

In [ ]:
holidays = data.loc[data['holiday'] == 1, 'timestamp'].dt.date.unique()
holidays

We add two temporary columns to compute the distance in days to or from each holiday.

In [ ]:
for i, hd in enumerate(holidays):
    data['hol_' + str(i)] = data['timestamp'].dt.date - hd

We are interested by the proximity to or from any holiday, so we want only the shortest gap:

In [ ]:
for i in range(data.shape[0]):
    if np.abs(data.loc[data.index[i], 'hol_0']) <= np.abs(data.loc[data.index[i], 'hol_1']):
        data.loc[data.index[i], 'gap_holiday'] = data.loc[data.index[i], 'hol_0']
    else:
        data.loc[data.index[i], 'gap_holiday'] = data.loc[data.index[i], 'hol_1']

In [ ]:
data['gap_holiday'] = data['gap_holiday'].astype('timedelta64[D]')

We no longer need the temporary columns:

In [ ]:
data.drop(['hol_0', 'hol_1'], axis=1, inplace=True)

Finally, we convert the timestamp into something easier to plot.

In [ ]:
data['t'] = (data['timestamp'].astype(np.int64)/1e11).astype(np.int64)
data.drop('timestamp', axis=1, inplace=True)
data

In [ ]:
cont_vars = ['value', 'hour_min', 'gap_holiday', 't']
cat_vars = ['day', 'month', 'day_of_week', 'holiday']

We now apply a Label Encoder to encode the categorical data from 0 to n, with n being the number of classes for the variable.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = [LabelEncoder() for _ in cat_vars] 
for col, enc in zip(cat_vars, label_encoders):
    data[col] = enc.fit_transform(data[col])
    
data

Notice that some of our data are continuous (or 'close to' continuous): `value`, `hour_min`, `gap_holiday` (even though this one only has a resolution of 1 day), `t`. Other variables are categorical: `day`, `month`, `day_week`, `holiday`. In a neural network model, these two types of data need to be handled differently, as we will see later.

It is time to split our data into a train set and a test set. Let's set 30% of the data aside.

In [ ]:
test_ratio = 0.3

In [ ]:
tr_idx = np.random.choice(np.arange(data.shape[0]), size=round(data.shape[0] * (1 - test_ratio)), replace=False)
tst_idx = list(set(range(data.shape[0])).difference(set(tr_idx)))
len(tr_idx), len(tst_idx)

In [ ]:
tr_data = data.iloc[tr_idx]
tst_data = data.iloc[tst_idx]

As we are using a neural network model, the continuous variables need to be normalised. This is important because the weights of a neural network are initialised at random from a common distribution, so all tend to have similar scales initially. Variables whose values spread are across different orders of magnitude would therefore cause serious difficulties when it comes to learning the optimal weights.

In [ ]:
scaler = preprocessing.StandardScaler().fit(tr_data[cont_vars])

In [ ]:
tr_data_scaled = tr_data.copy()
tr_data_scaled[cont_vars] = scaler.transform(tr_data[cont_vars])
tst_data_scaled = tst_data.copy()
tst_data_scaled[cont_vars] = scaler.transform(tst_data[cont_vars])

Note that the test data is normalised using parameters observed on the train set. Otherwise, we would be cheating!

In [ ]:
# tr_data_scaled = pd.DataFrame(tr_data_scaled, columns=tr_data.columns)
# tst_data_scaled = pd.DataFrame(tst_data_scaled, columns=tst_data.columns)

tr_data, tr_data_scaled

As you can see, the continuous variables are now standardised, while the categorical variables are unchanged. We can now save our data as CSV files and we are ready for the next phase.

In [ ]:
tr_data_scaled.to_csv(datasets_root/'train.csv', index=False)
tst_data_scaled.to_csv(datasets_root/'test.csv', index=False)

## 3. VAE Model
We first define a `Dataset` class that takes data from either the train set or the test set. It is also able to filter out some of the columns.

**Note**: For VAE training, we want to use the column `value` as a feature rather than a label, because we are doing unsupervised learning. In that case, we set the argument `lbl_as_feat` to True. If we were training a supervised model, then we would be able to use the same `Dataset` class but we would need to set `lbl_as_feat` to False, and the dataset would return `value` as a label.

In [ ]:
class TSDataset(Dataset):
    def __init__(self, split, cont_vars=None, cat_vars=None, lbl_as_feat=True):
        """
        split: 'train' if we want to get data from the training examples, 'test' for
        test examples, or 'both' to merge the training and test sets and return samples
        from either.
        cont_vars: List of continuous variables to return as features. If None, returns
        all continuous variables available.
        cat_vars: Same as above, but for categorical variables.
        lbl_as_feat: Set to True when training a VAE -- the labels (temperature values)
        will be included as another dimension of the data. Set to False when training
        a model to predict temperatures.
        """
        super().__init__()
        assert split in ['train', 'test', 'both']
        self.lbl_as_feat = lbl_as_feat
        if split == 'train':
            self.df = pd.read_csv(datasets_root/'train.csv')
        elif split == 'test':
            self.df = pd.read_csv(datasets_root/'test.csv')
        else:
            df1 = pd.read_csv(datasets_root/'train.csv')
            df2 = pd.read_csv(datasets_root/'test.csv')
            self.df = pd.concat((df1, df2), ignore_index=True)
        
        # Select continuous variables to use
        if cont_vars:
            self.cont_vars = cont_vars
            # If we want to use 'value' as a feature, ensure it is returned
            if self.lbl_as_feat:
                try:
                    assert 'value' in self.cont_vars
                except AssertionError:
                    self.cont_vars.insert(0, 'value')
            # If not, ensure it not returned as a feature
            else:
                try:
                    assert 'value' not in self.cont_vars
                except AssertionError:
                    self.cont_vars.remove('value')
                    
        else:  # if no list provided, use all available
            self.cont_vars = ['value', 'hour_min', 'gap_holiday', 't']
        
        # Select categorical variables to use
        if cat_vars:
            self.cat_vars = cat_vars
        else:  # if no list provided, use all available
            self.cat_vars = ['day', 'month', 'day_of_week', 'holiday']
        
        # Finally, make two Numpy arrays for continuous and categorical
        # variables, respectively:
        if self.lbl_as_feat:
            self.cont = self.df[self.cont_vars].copy().to_numpy(dtype=np.float32)
        else:
            self.cont = self.df[self.cont_vars].copy().to_numpy(dtype=np.float32)
            self.lbl = self.df['value'].copy().to_numpy(dtype=np.float32)
        self.cat = self.df[self.cat_vars].copy().to_numpy(dtype=np.int64)
            
    def __getitem__(self, idx):
        if self.lbl_as_feat:  # for VAE training
            return torch.tensor(self.cont[idx]), torch.tensor(self.cat[idx])
        else:  # for supervised prediction
            return torch.tensor(self.cont[idx]), torch.tensor(self.cat[idx]), torch.tensor(self.lbl[idx])
    
    def __len__(self):
        return self.df.shape[0]

Let's try to call from our dataset:

In [ ]:
ds = TSDataset(split='both', cont_vars=['value', 't'], cat_vars=['day_of_week', 'holiday'], lbl_as_feat=True)
print(len(ds))
it = iter(ds)
for _ in range(10):
    print(next(it))

As expected, with `lbl_as_feat==True`, we get two arrays for each sample: An array of continuous values represented by floats, and an array of categorical values represented by integers.

Now we move on to the fun part: The model itself. First, define individual modules. The core of our neural network will be a sequence of fully connected layers, whose number and sizes are passed through the hyperparameter `layer_dims` (a list of integers). In our experiment, we used `64,128,64`, i.e. 3 layers of dimensions 64, 128 and 64. Each layer can be batch-normalised. The input into the first layer is an aggregation of the continuous variables and embedding vectors encoding the categorical variables. 

Embedding vectors are a notion that is heavily used in Natural Language Processing. They are learnable vectors (of dimension 8 in our experiments) that express some non-explicit features of the variables. For instance, the vector for `day_of_week` can take 7 different sets of values (one for each day), each 8-dimensional. Depending on the value of this variable for each sample, the corresponding vector is retrieved in a lookup table, passed to the network, and updated during backpropagation. The vector for `day_of_week==0` could eventually capture the fact that the machine's activity is lower on such days, for instance. The important thing is that this learning is done without our intervention, and there is no easy way to interpret the learned embeddings.

In [ ]:
class Layer(nn.Module):
    '''
    A single fully connected layer with optional batch normalisation and activation.
    '''
    def __init__(self, in_dim, out_dim, bn = True):
        super().__init__()
        layers = [nn.Linear(in_dim, out_dim)]
        if bn: layers.append(nn.BatchNorm1d(out_dim))
        layers.append(nn.LeakyReLU(0.1, inplace=True))
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)

    
class Encoder(nn.Module):
    '''
    The encoder part of our VAE. Takes a data sample and returns the mean and the log-variance of the 
    latent vector's distribution.
    '''
    def __init__(self, hparams):
        super().__init__()

        self.embeds = nn.ModuleList([
            nn.Embedding(n_cats, emb_size) for (n_cats, emb_size) in hparams.embedding_sizes
        ])
        # The input to the first layer is the concatenation of all embedding vectors and continuous
        # values
        in_dim = sum(emb.embedding_dim for emb in self.embeds) + len(hparams.cont_vars)
        layer_dims = [in_dim] + [int(s) for s in hparams.layer_sizes.split(',')]
        bn = hparams.batch_norm
        self.layers = nn.Sequential(
            *[Layer(layer_dims[i], layer_dims[i + 1], bn) for i in range(len(layer_dims) - 1)],
        )
        self.mu = nn.Linear(layer_dims[-1], hparams.latent_dim)
        self.logvar = nn.Linear(layer_dims[-1], hparams.latent_dim)
    
    def forward(self, x_cont, x_cat):
        x_embed = [e(x_cat[:, i]) for i, e in enumerate(self.embeds)]        
        x_embed = torch.cat(x_embed, dim=1)
        x = torch.cat((x_embed, x_cont), dim=1)
        h = self.layers(x)
        mu_ = self.mu(h)
        logvar_ = self.logvar(h)
        return mu_, logvar_, x  # we return the concatenated input vector for use in loss fn
    

class Decoder(nn.Module):
    '''
    The decoder part of our VAE. Takes a latent vector (sampled from the distribution learned by the 
    encoder) and converts it back to a reconstructed data sample.
    '''
    def __init__(self, hparams):
        super().__init__()
#         self.final_activ = hparams.final_activ
        hidden_dims = [hparams.latent_dim] + [int(s) for s in reversed(hparams.layer_sizes.split(','))]
        out_dim = sum(emb_size for _, emb_size in hparams.embedding_sizes) + len(hparams.cont_vars)
        bn = hparams.batch_norm
        self.layers = nn.Sequential(
            *[Layer(hidden_dims[i], hidden_dims[i + 1], bn) for i in range(len(hidden_dims) - 1)],
        )
        self.reconstructed = nn.Linear(hidden_dims[-1], out_dim)
        
    def forward(self, z):
        h = self.layers(z)
        recon = self.reconstructed(h)
        return recon

Now for the full VAE, defined as a LightningModule:

In [ ]:
class VAE(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        if isinstance(hparams, dict):
            hparams = Namespace(**hparams)
        self.hparams = hparams
        self.encoder = Encoder(hparams)
        self.decoder = Decoder(hparams)
        self.stdev = hparams.stdev
        self.kld_beta = hparams.kld_beta
        self.lr = hparams.lr
        self.wd = hparams.weight_decay
        
    def reparameterize(self, mu, logvar):
        '''
        The reparameterisation trick allows us to backpropagate through the encoder.
        '''
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std) * self.stdev
            return eps * std + mu
        else:
            return mu
        
    def forward(self, batch):
        x_cont, x_cat = batch
        assert x_cat.dtype == torch.int64
        mu, logvar, x = self.encoder(x_cont, x_cat)
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z)
        return recon, mu, logvar, x
        
    def loss_function(self, obs, recon, mu, logvar):
        recon_loss = F.smooth_l1_loss(recon, obs, reduction='sum')
        kld = -0.5 * torch.sum(1 + logvar - mu ** 2 - logvar.exp())
        return recon_loss, kld
                               
    def training_step(self, batch, batch_idx):
        recon, mu, logvar, x = self.forward(batch)
        # The loss function compares the concatenated input vector including
        # embeddings to the reconstructed vector
        recon_loss, kld = self.loss_function(x, recon, mu, logvar)
        loss = recon_loss + self.kld_beta * kld

        self.log('total_tr_loss', loss.mean(dim=0), on_step=True, prog_bar=True, 
                 logger=True)
        self.log('recon_loss', recon_loss.mean(dim=0), on_step=True, prog_bar=True, 
                 logger=True)
        self.log('kld', kld.mean(dim=0), on_step=True, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        recon, mu, logvar, x = self.forward(batch)
        recon_loss, kld = self.loss_function(x, recon, mu, logvar)
        loss = recon_loss + self.kld_beta * kld
        self.log('test_loss', loss)
        return loss
        
    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.lr, 
                                weight_decay=self.hparams.weight_decay, 
                                eps=1e-4)
        sch = torch.optim.lr_scheduler.MultiplicativeLR(opt, lr_lambda=lambda epoch: 0.95)
        return opt
    
    def train_dataloader(self):
        dataset = TSDataset('train', cont_vars=self.hparams.cont_vars, 
                            cat_vars = self.hparams.cat_vars, lbl_as_feat=True)
        return DataLoader(dataset, batch_size=self.hparams.batch_size, num_workers=8)
    
    def test_dataloader(self):
        dataset = TSDataset('test', cont_vars=self.hparams.cont_vars,
                            cat_vars=self.hparams.cat_vars, lbl_as_feat=True)
        return DataLoader(dataset, batch_size=self.hparams.batch_size, num_workers=8)        

Note that different reconstruction loss functions are possible, the most obvious being the Mean Square Error (MSE). However, it is quite sensitive to outliers. In our case, almost by definition, we expect outliers to be present so it might be useful to use a loss function that is less sensitive to them. We use the Huber loss with $\delta=1$ (called `smooth_l1_loss` in Pytorch).

Let's define the model and data hyperparameters. Note that `stdev` (the standard deviation of the normal distribution from which we sample $\epsilon$ in the `reparameterize()` method) would typically be 1. However, several papers found improvements with smaller values, such as 1e-1. This is what we are going to use, as it gives us lower loss values.

Another hyperparameter worth mentioning is `kld_beta`, the coefficient applied to the KLD loss term in the total loss computation (this hyperparameter means that our VAE is technically a $\beta$-VAE). This helps because the reconstruction loss is typically harder to improve than the KLD loss, therefore if both were weighted equally, the model would start by optimising the KLD loss before improving the reconstruction loss substantially. By setting this coefficient to a value < 1, we try to make sure that both loss values improve at a similar rate.

In [ ]:
cont_features = ['value', 'hour_min', 'gap_holiday', 't'] 
cat_features = ['day', 'month', 'day_of_week', 'holiday'] 

embed_cats = [len(tr_data_scaled[c].unique()) for c in cat_features]

hparams = OrderedDict(
    run='all_vars_embsz8_latsz32_bsz128_lay64-128-64_ep100',
    cont_vars = cont_features,
    cat_vars = cat_features,
    embedding_sizes = [(embed_cats[i], 8) for i in range(len(embed_cats))],
    latent_dim = 32,
    layer_sizes = '64,128,64',
    batch_norm = True,
    stdev = 0.1,
    kld_beta = 0.2,
    lr = 0.0005,
    weight_decay = 1e-5,
    batch_size = 128,
    epochs = 200,
)

In [ ]:
hparams

In [ ]:
from argparse import Namespace
# Simulate a Namespace with the defined hyperparameters
hparams = Namespace(**hparams)

First, let's choose our learning rate. Instead of searching ourselves, we can leverage a functionality provided by Pytorch-Lightning that automates the search for this hyperparameter.

EDIT: Following an update to Pytorch-Lightning, this functionality return a value that is too high and causes the loss to diverge. Therefore, we set the loss manually to 0.001. Note that we also use gradient clipping to further enforce convergence.

In [ ]:
!rm /kaggle/working/vae_weights*

In [ ]:
torch.cuda.empty_cache()
model = VAE(hparams)
logger = WandbLogger(name=hparams.run, project='VAE_Anomaly', version=hparams.run)
ckpt_callback = pl.callbacks.ModelCheckpoint(dirpath='.', filename='vae_weights')
# Replace argument logger by None if you don't have a WandB account (and don't want to create one)
trainer = pl.Trainer(gpus=-1, logger=logger, max_epochs=hparams.epochs, 
                     auto_lr_find=False, benchmark=True, callbacks=[ckpt_callback],
                     gradient_clip_val=0.5
                     )
# trainer.tune(model)

In [ ]:
trainer.fit(model)

Training takes only a few minutes on GPU. Let's see how this model performs on the test set:

In [ ]:
trainer.test()

Hopefully, we should find a test loss that is fairly close to the training loss, indicating that we are not overfitting. A good way to regularise training would be creating a separate validation set and stopping the training early when the loss computed on that set starts to increase (or stops decreasing). Pytorch-Lightning makes early stopping extremely easy to implement: once you have created the validation set, it is just a case of passing an additional argument to the Trainer class. Note also that we could probably improve on these results by further optimising hyperparameters (WandB actually offers an easy framework for running hyperparameter optimisation by random, grid or Bayesian search). For now, let's go with these weights and see if the model can find outliers.

In [ ]:
trained_model = VAE.load_from_checkpoint('./vae_weights-v1.ckpt')
trained_model.freeze()
dataset = TSDataset('test', cont_vars=hparams.cont_vars, 
                    cat_vars=hparams.cat_vars,
                    lbl_as_feat=True) 
losses = []
# run predictions for the training set examples
for i in range(len(dataset)):
    x_cont, x_cat = dataset[i]
    x_cont.unsqueeze_(0)
    x_cat.unsqueeze_(0)
    recon, mu, logvar, x = trained_model.forward((x_cont, x_cat))
    recon_loss, kld = trained_model.loss_function(x, recon, mu, logvar)
    losses.append(recon_loss + trained_model.hparams.kld_beta * kld)
    
data_with_losses = dataset.df
data_with_losses['loss'] = np.asarray(losses)
data_with_losses.sort_values('t', inplace=True)
data_with_losses.head()

How are loss values distributed?

In [ ]:
mean, sigma = data_with_losses['loss'].mean(), data_with_losses['loss'].std()
mean, sigma

Let's define a threshold value (in multiples of the standard deviation) beyond which instances are classified as anomalies:

In [ ]:
thresh = 6 # threshold for anomaly, in multiples of sigma.

We can now flag as anomalies any instance where the loss is very far from the centre of the distribution:

In [ ]:
data_with_losses['anomaly'] = data_with_losses['loss'] > (mean + sigma * thresh)
print(data_with_losses.head())
colors = ['red' if anomaly else 'blue' for anomaly in data_with_losses['anomaly']]

Let's see this in a histogram. Red colour denotes abnormal samples.

In [ ]:
anomalies_loss = data_with_losses.loc[data_with_losses['anomaly'], 'loss']
normals_loss   = data_with_losses.loc[~data_with_losses['anomaly'], 'loss']
plt.hist([normals_loss, anomalies_loss], bins=100, stacked=True, color=['blue', 'red'], label=['normal', 'abnormal'])
plt.legend();

What does it mean in terms of the distribution of the temperature values? We first "unscale" the data to put each feature back into the value ranges of the raw dataset.

In [ ]:
data_with_losses_unscaled = data_with_losses.copy()
data_with_losses_unscaled[cont_vars] = scaler.inverse_transform(data_with_losses[cont_vars])
for enc, var in zip(label_encoders, cat_vars):
    data_with_losses_unscaled[var] = enc.inverse_transform(data_with_losses[var])
data_with_losses_unscaled = pd.DataFrame(data_with_losses_unscaled, columns=data_with_losses.columns)
print(data_with_losses_unscaled.head())

In [ ]:
data_with_losses_unscaled

Then we build the histogram of `values`, coloured depending on whether or not the loss value for the corresponding observations are considered abnormal or not. 

In [ ]:
anomalies_value = data_with_losses_unscaled.loc[data_with_losses_unscaled['anomaly'], ['loss','value']]
normals_value = data_with_losses_unscaled.loc[~data_with_losses_unscaled['anomaly'], ['loss','value']]

plt.hist([normals_value['value'], anomalies_value['value']], bins=100, stacked=True, color=['blue', 'red'], label=['normal', 'abnormal'])
plt.legend();

![](http://)It's a little hard to tell on this figure, but some of the instances flagged as abnormal are not at the extreme end of the temperature spectrum, as they would be using a more naive approach based only on the temperature. Let see these flagged examples on the original timeseries:


In [ ]:
anomalies_ts = data_with_losses_unscaled.loc[data_with_losses_unscaled['anomaly'], ('t', 'value')]
holidays_ts = data_with_losses_unscaled.loc[data_with_losses_unscaled['holiday'] == 1, ('t', 'value')]
print(anomalies_ts.head())
fig, ax = plt.subplots()
ax.plot(data_with_losses_unscaled['t'], data_with_losses_unscaled['value'], color='blue')
ax.scatter(anomalies_ts['t'], anomalies_ts['value'], color='red', label='anomaly')
plt.legend()
plt.show();

[](http://)With a threshold at 6 sigmas, our model detects two of the three anomalies (it misses the second one), and does not generate too many false positives. The threshold that we defined can be adjusted to favour precision or recall and the optimal value could be determined by the area under the precision-recall curve. It would also be interesting to perform an ablation study to understand which of the engineered features are the most helpful.

I hope this notebook was informative and gave you some insight into VAEs and their usefulness!

## References
\[1\] Doersch, C. (2016). Tutorial on variational autoencoders. arXiv preprint arXiv:1606.05908.  
\[2\] Ahmad, S., Lavin, A., Purdy, S., & Agha, Z. (2017). Unsupervised real-time anomaly detection for streaming data. Neurocomputing, 262, 134-147.